In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Changing the Directory to the folder where we have the files required for ternarising the neural network


In [2]:
cd /content/drive/My\ Drive/NeuralnetProject

/content/drive/My Drive/NeuralnetProject


Importing the required libraries and  packages

In [3]:


from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, MaxPooling2D, Conv2D
from keras.layers import Flatten
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
import keras.backend as K
from ternary_ops import ternarize
from ternary_layers import TernaryDense, TernaryConv2D

from matplotlib import pyplot as plt

Using TensorFlow backend.


In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from sklearn.model_selection import train_test_split

Loading the Data 

In [0]:
(X_train,Y_train),(X_test,Y_test)= mnist.load_data()

In [6]:
X_train.shape

(60000, 28, 28)

In [0]:
Y_train=keras.utils.to_categorical(Y_train, 10)
Y_test=keras.utils.to_categorical(Y_test, 10)


Reshaping the data

In [0]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [9]:
X_test.shape

(10000, 28, 28, 1)

Analysing the result of alexnet on mnist data  before ternarising the weights

In [10]:
model=keras.Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3),
                 padding='same',activation='relu'))

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))   
model.add(Dense(128, activation='relu'))
model.add(Dropout(.5))  
           
model.add(Dense(10, activation='softmax'))

In [11]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [12]:
batch_size = 128
epochs = 15
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data = (X_test, Y_test))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/15





60000/60000 [==============================] - 7s 120us/step - loss: 0.3818 - acc: 0.8832 - val_loss: 0.0787 - val_acc: 0.9769
Epoch 2/15
60000/60000 [==============================] - 5s 78us/step - loss: 0.0744 - acc: 0.9808 - val_loss: 0.0537 - val_acc: 0.9851
Epoch 3/15
60000/60000 [==============================] - 5s 78us/step - loss: 0.0496 - acc: 0.9874 - val_loss: 0.0492 - val_acc: 0.9874
Epoch 4/15
60000/60000 [==============================] - 5s 78us/step - loss: 0.0369 - acc: 0.9904 - val_loss: 0.0338 - val_acc: 0.9915
Epoch 5/15
60000/60000 [==============================] - 5s 76us/step - loss: 0.0261 - acc: 0.9931 - val_loss: 0.0410 - val_acc: 0.9906
Epoch 6/15
60000/60000 [==================

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 128)       1280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          73792     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 1, 32)         

This has a total of 300074 parameters so it requires 300074 * 32 bits for storing them 

Ternazing 
Reference:
https://github.com/DingKe/nn_playground/tree/master/ternarynet

In [0]:
(X_train,Y_train),(X_test,Y_test)= mnist.load_data()

In [15]:
X_train = X_train.reshape(60000,28, 28,1)
X_test = X_test.reshape(10000,28, 28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, 10) * 2 - 1 # -1 or 1 for hinge loss
Y_test = np_utils.to_categorical(Y_test, 10) * 2 - 1


60000 train samples
10000 test samples


In [16]:
H = 1.
kernel_lr_multiplier = 'Glorot'
use_bias = False
epsilon = 1e-6
def ternary_tanh(x):
    x = K.clip(x, -1, 1)
    return ternarize(x)
momentum = 0.9


# dropout
p1 = 0.25
p2 = 0.5
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / 250)
ternmodel = Sequential()

ternmodel.add(TernaryConv2D(128, kernel_size=(3, 3), input_shape=(28,28,1),
                        H=H, kernel_lr_multiplier=kernel_lr_multiplier, 
                        use_bias=use_bias, name='conv1'))
ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn1'))

ternmodel.add(Activation(ternary_tanh))
ternmodel.add(MaxPooling2D(pool_size=(2,2)))

ternmodel.add(TernaryConv2D(128, kernel_size=(3, 3), H=H, kernel_lr_multiplier=kernel_lr_multiplier, 
                         use_bias=use_bias, name='conv2'))
ternmodel.add(Activation(ternary_tanh))

ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn2'))
ternmodel.add(MaxPooling2D(pool_size=(2, 2), name='pool2'))
ternmodel.add(TernaryConv2D(128, kernel_size=(3, 3), H=H, kernel_lr_multiplier=kernel_lr_multiplier, 
                         padding='same',use_bias=use_bias, name='conv3'))
ternmodel.add(Activation(ternary_tanh))

ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn3'))



ternmodel.add(TernaryConv2D(64, kernel_size=(3, 3), H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                        use_bias=use_bias, name='conv4'))
ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn4'))
ternmodel.add(Activation(ternary_tanh))

ternmodel.add(TernaryConv2D(64, kernel_size=(3, 3), H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                        padding='same', use_bias=use_bias, name='conv5'))

ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn5'))
ternmodel.add(Activation(ternary_tanh))
ternmodel.add(TernaryConv2D(32, kernel_size=(3, 3), H=H, kernel_lr_multiplier=kernel_lr_multiplier,
                        use_bias=use_bias, name='conv6'))

ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, axis=1, name='bn6'))
ternmodel.add(Activation(ternary_tanh))
ternmodel.add(Flatten())
# dense1
ternmodel.add(TernaryDense(128, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias, name='dense1'))
ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn7'))
ternmodel.add(Activation(ternary_tanh))
ternmodel.add(TernaryDense(128, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias, name='dense2'))
ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn8'))
ternmodel.add(Activation(ternary_tanh))
ternmodel.add(TernaryDense(128, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias, name='dense3'))
ternmodel.add(BatchNormalization(epsilon=epsilon, momentum=momentum, name='bn9'))
ternmodel.add(Activation(ternary_tanh))

ternmodel.add(TernaryDense(10, H=H, kernel_lr_multiplier=kernel_lr_multiplier, use_bias=use_bias, name='dense4'))
ternmodel.add(Activation('softmax'))


In [17]:
opt = Adam(lr=lr_start) 
ternmodel.compile(loss='squared_hinge', optimizer=opt, metrics=['acc'])
ternmodel.summary()

lr_scheduler = LearningRateScheduler(lambda e: lr_start * lr_decay ** e)
ternmodel.fit(X_train, Y_train,
                    batch_size=128, epochs=15,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[lr_scheduler])
score = ternmodel.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (TernaryConv2D)        (None, 26, 26, 128)       1152      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 26, 26, 128)       104       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 128)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2 (TernaryConv2D)        (None, 11, 11, 128)       147456    
_________________________________________________________________
activation_2 (Activation)    (None, 11, 11, 128)       0         
_________________________________________________________________
bn2 (BatchNormalization)     (None, 11, 11, 128)      

In [0]:
ternmodel.save('ternmodel.h5')

In [0]:
weights= ternmodel.layers[4].get_weights()

We can observe that the weights are still in floating point even after using the method given by Dingke numbers so I have changed these weights to ternary weights(-1,0,1) which are easy for implementation on embedded devices.

In [20]:
weights

[array([[[[-0.7867752 ,  0.40090546,  0.6336262 , ...,  0.06273282,
           -0.22798817,  0.6620213 ],
          [ 0.99389   , -0.97301275,  0.37504727, ..., -0.42001793,
            0.4767717 , -0.5886443 ],
          [-0.30543464, -0.39206964, -0.4234336 , ..., -0.8125192 ,
           -0.25350738,  0.6411867 ],
          ...,
          [-0.8997264 ,  0.53432214, -0.24971619, ...,  0.121704  ,
            0.6676413 , -0.61509836],
          [ 0.904689  , -0.4937072 , -0.3781922 , ..., -0.89181095,
           -0.5968728 , -0.8937968 ],
          [-0.32702166, -0.39226657, -0.2018477 , ...,  0.55474955,
            0.09734698,  0.2050197 ]],
 
         [[-0.9123636 , -0.36988607, -0.54369926, ..., -0.9135784 ,
            0.15435064, -0.9920048 ],
          [ 0.12522647, -0.25335905, -0.40751013, ...,  0.29138437,
           -0.613593  ,  0.7347691 ],
          [-0.18452479,  0.8434135 ,  0.6372471 , ..., -0.73786724,
            0.9690748 ,  0.8618747 ],
          ...,
          [ 0

Ternarising the weights of the neural network. This function below converts the weights of a 2D conv layer and Dense layer to ternary weights when passed the list of layer numbers along with the type of the layers whether ConV2D or Dense. When Type is Dense the list passed along with it should contain the layer numbers of Dense layers only. 

In [0]:
def Ternarize(a,listofnum):
  if(a=='ConV2D'):
    for convlayer in listofnum:
      weights= ternmodel.layers[convlayer].get_weights()
      for w in weights[0]:
        for i in w:
          for j in i:
            for k in range(len(j)):
              if j[k]>0.5:
                j[k]=1
              elif j[k]<=-0.5:
                j[k]=-1
              else:
                j[k]=0
      ternmodel.layers[convlayer].set_weights(weights)
  if(a=='Dense'):
    for denselayer in listofnum:
      denseweights= ternmodel.layers[denselayer].get_weights()
      for weight in denseweights[0]:
        for l in range(len(weight)):
          if weight[l]>0.5:
            weight[l]=1
          elif weight[l]<=-0.5:
            weight[l]=-1
          else:
            weight[l]=0
      ternmodel.layers[denselayer].set_weights(denseweights)






In [0]:
Ternarize('ConV2D',[0,4,8,11,14,17])

In [0]:
Ternarize('Dense',[21,24,27,30])

In [24]:
ternmodel.layers[17].get_weights()

[array([[[[ 0.,  1.,  0., ..., -1.,  1.,  0.],
          [ 0., -1.,  1., ...,  0.,  1., -1.],
          [ 0.,  0.,  1., ...,  0.,  1.,  0.],
          ...,
          [ 0.,  0.,  1., ...,  0.,  1.,  0.],
          [ 0.,  0.,  1., ...,  0., -1.,  1.],
          [ 0., -1., -1., ...,  0., -1.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0., -1., -1.],
          [ 0.,  1.,  0., ..., -1.,  0.,  0.],
          [ 1.,  0.,  0., ...,  0.,  1.,  0.],
          ...,
          [ 1.,  1.,  0., ...,  0.,  0.,  0.],
          [ 1., -1., -1., ...,  1.,  0.,  0.],
          [ 1.,  0., -1., ...,  0., -1.,  1.]],
 
         [[-1.,  0.,  0., ..., -1., -1.,  1.],
          [ 1.,  1., -1., ...,  1., -1.,  0.],
          [-1.,  0.,  1., ..., -1.,  1.,  0.],
          ...,
          [-1.,  1.,  1., ...,  0.,  0.,  1.],
          [ 0., -1., -1., ...,  1.,  0.,  0.],
          [-1.,  0.,  0., ...,  0., -1.,  0.]]],
 
 
        [[[ 1.,  0.,  0., ..., -1., -1., -1.],
          [ 0.,  0.,  0., ...,  0.,  0.,  1.],
   

In [25]:
ternmodel.layers[24].get_weights()

[array([[ 1., -1.,  1., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  1.,  0.,  1.],
        [ 0.,  0.,  1., ...,  0.,  1.,  1.],
        ...,
        [ 0.,  0.,  1., ...,  1.,  0.,  0.],
        [ 1.,  1.,  0., ...,  1., -1., -1.],
        [ 0.,  0.,  1., ..., -1., -1., -1.]], dtype=float32)]

In [26]:
ternmodel.layers[30].get_weights()

[array([[-1.,  0., -1., ..., -1., -1.,  0.],
        [ 1., -1.,  1., ...,  0.,  0.,  1.],
        [ 0.,  1.,  0., ..., -1., -1.,  0.],
        ...,
        [ 1., -1.,  0., ..., -1.,  0., -1.],
        [-1.,  0.,  0., ...,  0.,  0., -1.],
        [ 0.,  0., -1., ...,  0.,  1.,  1.]], dtype=float32)]

These ternary weights are easy for implementation on embedded devices.
Ternary weight network's achieve up to 16× or 32× model compression rate compared with the float (32-bit) or double
(64-bit) precision counterparts
Reference:https://arxiv.org/pdf/1605.04711.pdf

In [0]:
ternmodel.save('ternweights.h5')

In [0]:
ternmodel.load_weights('ternweights.h5') 

In [29]:
score = ternmodel.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.9243518768310547
Test accuracy: 0.9367


We can se that before and after converting weights to ternary the test accuracy is the same. This model has a total of 464964 parameters which require only 2 bits for storage. which is very less compared to before ternarising the network.